# Create pyCIAM Storm Costs Lookup Table

Calculating the storm costs in a CIAM model involves a numerical integration over both elevation and the quantiles of storm surge at each segment-ADM1 location. This is too computationally intensive to run for all seg-ADMs for each year for all SLR trajectories, especially when using pyCIAM to run a Monte Carlo analysis across tens of thousands of SLR trajectories. Instead, we build a lookup table indexed by seg-ADM, LSLR, adaptation type (retreat vs. protect), cost type (mortality vs. capital loss), and `rhdiff` (the difference between the retreat/protect height and lslr). This is similar to how it is treated in the original CIAM model except that:

1. We use a lookup table rather than a parameterized exponential function of `rhdiff` and `lslr`
2. We account for elevational heterogeneity in population and capital when evaluating our costs in retreat scenarios. The original CIAM included `lslr` in their exponential function only for the protect adaptation type, while for `noAdaptation` and `retreat`, the function was only of `rhdiff`.

## Setup

In [1]:
import distributed as dd
import pandas as pd
from pyCIAM.surge import damage_funcs
from pyCIAM.surge.lookup import create_surge_lookup
from shared import (
    PATH_PARAMS,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    start_dask_cluster,
)

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [3]:
SEG_CHUNKSIZE = 50

PARAMS = pd.read_json(PATH_PARAMS)["values"]

In [4]:
DMF_I = getattr(damage_funcs, PARAMS.dmf + "_i")
DDF_I = getattr(damage_funcs, PARAMS.ddf + "_i")

In [5]:
client, cluster = start_dask_cluster()
cluster

LocalCluster(2e6d582f, 'tcp://127.0.0.1:46337', workers=4, threads=16, memory=124.37 GiB)

## Run surge damage calculations for each combo

In [6]:
futs = {}
for kind, sliiders in [("seg_adm", PATH_SLIIDERS), ("seg", PATH_SLIIDERS_SEG)]:
    futs[kind] = create_surge_lookup(
        sliiders,
        [PATH_SLR_INT],
        PATHS_SURGE_LOOKUP[kind],
        kind,
        PARAMS.at_start,
        PARAMS.n_interp_pts_lslr,
        PARAMS.n_interp_pts_rhdiff,
        DDF_I,
        DMF_I,
        quantiles=[0.5],
        start_year=PARAMS.model_start,
        slr_0_years=PARAMS.slr_0_year,
        client=client,
        force_overwrite=True,
        seg_chunksize=SEG_CHUNKSIZE,
        mc_dim="quantile",
    )

In [35]:
[i for i in futs["seg_adm"] if i.status == "error"], len([i for i in futs["seg_adm"] if i.status == "finished"]), len(futs["seg_adm"])

([], 68, 2396)

## Close

In [7]:
# ensure completion and close cluster
all_futs = futs["seg"] + futs["seg_adm"]
dd.wait(all_futs)
assert [f.status == "finished" for f in all_futs]
finished = True

In [8]:
client.cluster.close(), client.close()

(None, None)